# Using TensorFlow Recommenders with TFX

***A tutorial to train a TensorFlow Recommenders ranking model as a [TFX pipeline](https://www.tensorflow.org/tfx).***

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/examples/ranking_tfx"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/ranking_tfx.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/ranking_tfx.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/ranking_tfx.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.6.2
TFX version: 1.3.4


In [2]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

### Set up variables

There are some variables used to define a pipeline. You can customize these
variables as you want. By default all output from the pipeline will be
generated under the current directory. Instead of using the SchemaGen component to generate a schema, for this
tutorial we will create a hardcoded schema.

In [3]:
import os

PIPELINE_NAME = 'TFRS-ranking'
WORKING_DIR = 'bachelor_2022/artifact'
PIPE_DIR = 'pipeline'

# Directory where MovieLens 100K rating data lives
DATA_ROOT = os.path.join('data', PIPELINE_NAME)
print(DATA_ROOT)
# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join(PIPE_DIR, 'pipelines', PIPELINE_NAME)
print(PIPELINE_ROOT)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join(PIPE_DIR, 'metadata', PIPELINE_NAME, 'metadata.db')
print(METADATA_PATH)
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join(PIPE_DIR, 'serving_model', PIPELINE_NAME)
print(SERVING_MODEL_DIR)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

data/TFRS-ranking
pipeline/pipelines/TFRS-ranking
pipeline/metadata/TFRS-ranking/metadata.db
pipeline/serving_model/TFRS-ranking


In [4]:
print(DATA_ROOT)
print(PIPELINE_ROOT)

data/TFRS-ranking
pipeline/pipelines/TFRS-ranking


### Write model training code

We will build a simple ranking model to predict movie ratings. This model training code will be saved to a separate file.

In this tutorial we will use
[Generic Trainer](https://www.tensorflow.org/tfx/guide/trainer#generic_trainer)
of TFX which support Keras-based models. You need to write a Python file
containing `run_fn` function, which is the entrypoint for the `Trainer`
component.

In [5]:
_trainer_module_file = 'tfrs_ranking_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import Dict, Text
from typing import List

import numpy as np
import tensorflow as tf

from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_recommenders as tfrs
from tensorflow_transform.tf_metadata import schema_utils
from tfx import v1 as tfx
from tfx_bsl.public import tfxio

_FEATURE_KEYS = ['userId', 'movieId']
_LABEL_KEY = 'rating'

_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
        for feature in _FEATURE_KEYS
    }, _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    unique_user_ids = np.array(range(943)).astype(str)
    unique_movie_ids = np.array(range(1682)).astype(str)

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,), name='userId', dtype=tf.int64),
        tf.keras.layers.Lambda(lambda x: tf.as_string(x)),
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(
            len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,), name='movieId', dtype=tf.int64),
        tf.keras.layers.Lambda(lambda x: tf.as_string(x)),
        tf.keras.layers.StringLookup(
            vocabulary=unique_movie_ids, mask_token=None),
        tf.keras.layers.Embedding(
            len(unique_movie_ids) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

  def call(self, inputs):

    user_id, movie_id = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_id)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=2))


class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()])

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model((features['userId'], features['movieId']))

  def compute_loss(self,
                   features: Dict[Text, tf.Tensor],
                   training=False) -> tf.Tensor:

    labels = features[1]
    rating_predictions = self(features[0])

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 256) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  return MovielensModel()


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files, fn_args.data_accessor, schema, batch_size=8192)
  eval_dataset = _input_fn(
      fn_args.eval_files, fn_args.data_accessor, schema, batch_size=4096)

  model = _build_keras_model()

  model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      epochs = 3,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  model.save(fn_args.serving_model_dir)

Overwriting tfrs_ranking_trainer.py


Now you have completed all preparation steps to build the TFX pipeline.

### Write a pipeline definition

We define a function to create a TFX pipeline. A `Pipeline` object
represents a TFX pipeline which can be run using one of pipeline
orchestration systems that TFX supports.


In [6]:
CONTEXT_PIPE_NAME = "tfx_tfrs_debug"

context = InteractiveContext(
    pipeline_root='/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/' + CONTEXT_PIPE_NAME,
    pipeline_name= CONTEXT_PIPE_NAME
)

In [7]:
example_gen = tfx.components.CsvExampleGen(input_base=DATA_ROOT)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 7
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:1979205,xor_checksum:1654179951,sum_checksum:1654179951"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.4"
          }
        }
        state: LIVE
        create_time_since_epoch: 1654684401588
        last_update_time_since_epoch: 1654684414501
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [8]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/CsvExampleGen/examples/1


In [9]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=False)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.


INFO:absl:Statistics for split train written to /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/StatisticsGen/statistics/8/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/StatisticsGen/statistics/8/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 7
        type_id: 16
        uri: "/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/StatisticsGen/statistics/8"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.4"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [10]:
context.show(statistics_gen.outputs['statistics'])

In [12]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/SchemaGen/schema/9/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/SchemaGen/schema/9"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.4"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'movieId',INT,required,,-
'rating',INT,required,,-
'timestamp',INT,required,,-
'userId',INT,required,,-


In [14]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/ExampleValidator/anomalies/10/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/ExampleValidator/anomalies/10/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 10
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 9
        type_id: 20
        uri: "/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/ExampleValidator/anomalies/10"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.4"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
context.show(example_validator.outputs['anomalies'])

/opt/miniconda3/envs/rs_pipe/lib/python3.6/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [16]:
trainer = tfx.components.Trainer(
    module_file=_trainer_module_file,
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=12),
    eval_args=tfx.proto.EvalArgs(num_steps=24))

context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/home/stud/s/ss490/share/bachelor_2022/artifact/tfrs_ranking_trainer.py' (including modules: ['tfrs_ranking_trainer']).
INFO:absl:User module package has hash fingerprint version bc2b229701436196f12efdbb4e880e3fa05103599208d98a05325f064775f21c.
INFO:absl:Executing: ['/opt/miniconda3/envs/rs_pipe/bin/python', '/tmp/tmpopq1ivxx/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpgueixvfq', '--dist-dir', '/tmp/tmpxznw53mm']
INFO:absl:Successfully built user code wheel distribution at '/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/_wheels/tfx_user_code_Trainer-0.0+bc2b229701436196f12efdbb4e880e3fa05103599208d98a05325f064775f21c-py3-none-any.whl'; target user module is 'tfrs_ranking_trainer'.
INFO:absl:Full user module path is 'tfrs_ranking_trainer@/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/_wheels/tfx_user_code_Trainer-0.0+bc2b229701436196f12efdbb4e880e3fa05103599208d98a053

Epoch 1/3
12/12 [==============================] - 5s 303ms/step - root_mean_squared_error: 2.1042 - loss: 4.1821 - regularization_loss: 0.0000e+00 - total_loss: 4.1821 - val_root_mean_squared_error: 1.1206 - val_loss: 1.2608 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.2608
Epoch 2/3
12/12 [==============================] - 4s 327ms/step - root_mean_squared_error: 1.1140 - loss: 1.2402 - regularization_loss: 0.0000e+00 - total_loss: 1.2402 - val_root_mean_squared_error: 1.1118 - val_loss: 1.2126 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.2126
Epoch 3/3
12/12 [==============================] - 3s 235ms/step - root_mean_squared_error: 1.1086 - loss: 1.2263 - regularization_loss: 0.0000e+00 - total_loss: 1.2263 - val_root_mean_squared_error: 1.0971 - val_loss: 1.1853 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.1853


INFO:tensorflow:Assets written to: /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/Trainer/model/11/Format-Serving/assets


INFO:absl:Training complete. Model written to /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/Trainer/model/11/Format-Serving. ModelRun written to /home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/Trainer/model_run/11
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 11
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 10
        type_id: 22
        uri: "/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/Trainer/model/11"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.4"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 11
        type_id: 23
        uri: "/home/stud/s/ss490/share/bachelor_2022/artifact/pipeline/tfx_tfrs_debug/Trainer/model_run/11"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.4"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=True)

INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 12
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [80]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            #signature_name='serving_default',
            label_key='rating'
            #preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='Precision',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
             ]
        )
    ],
    slicing_specs=[
        tfma.SlicingSpec()
    ])

In [54]:
# eval_config = tfma.EvalConfig(
#       model_specs=[tfma.ModelSpec(label_key='rating')],
#       slicing_specs=[
#           # An empty slice spec means the overall slice, i.e. the whole dataset.
#           tfma.SlicingSpec(),
#           # Calculate metrics for each penguin species.
#           tfma.SlicingSpec(feature_keys=['rating']),
#           ],
#       metrics_specs=[
#           tfma.MetricsSpec(per_slice_thresholds={
#               'Precision':
#                   tfma.PerSliceMetricThresholds(thresholds=[
#                       tfma.PerSliceMetricThreshold(
#                           slicing_specs=[tfma.SlicingSpec()],
#                           threshold=tfma.MetricThreshold(
#                               value_threshold=tfma.GenericValueThreshold(
#                                    lower_bound={'value': 0.1}),
#                               # Change threshold will be ignored if there is no
#                               # baseline model resolved from MLMD (first run).
#                               change_threshold=tfma.GenericChangeThreshold(
#                                   direction=tfma.MetricDirection.HIGHER_IS_BETTER,
#                                   absolute={'value': -1e-10}))
#                        )]),
#           })],
#       )

In [74]:
# from google.protobuf import text_format

# eval_config = text_format.Parse("""
#   ## Model information
#   model_specs {
#     # For keras (and serving models) we need to add a `label_key`.
#     label_key: "rating"
#   }

#   ## Post training metric information. These will be merged with any built-in
#   ## metrics from training.
#   metrics_specs {
#     metrics { class_name: "ExampleCount" }
#     metrics { class_name: "BinaryAccuracy" }
#     metrics { class_name: "BinaryCrossentropy" }
#     metrics { class_name: "AUC" }
#     metrics { class_name: "AUCPrecisionRecall" }
#     metrics { class_name: "Precision" }
#     metrics { class_name: "Recall" }
#     metrics { class_name: "MeanLabel" }
#     metrics { class_name: "MeanPrediction" }
#     metrics { class_name: "Calibration" }
#     metrics { class_name: "CalibrationPlot" }
#     metrics { class_name: "ConfusionMatrixPlot" }
#     # ... add additional metrics and plots ...
#   }

#   ## Slicing information
#   slicing_specs {}  # overall slice
#   slicing_specs {
#     feature_keys: ["movieId"]
#   }
# """, tfma.EvalConfig())

In [78]:
# eval_config = tfma.EvalConfig(
#     model_specs=[
#       tfma.ModelSpec(model_type=tfma.constants.TF_KERAS, label_key="rating")
#     ],
#     slicing_specs=[
#         tfma.SlicingSpec()
#     ]
# )

In [81]:
evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "Precision",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "rating",\n      "preprocessing_function_names": [\n        "transform_features"\n      ],\n      "signature_name": "serving_default"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "movieId",\n        "userId"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator

ValueError: transform_features not found in model signatures: _SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, userId, movieId) at 0x7F4C452FCBE0>}) [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/ExtractTransformedFeatures/Predict']

In [44]:
context.show(evaluator.outputs['evaluation'])

ValueError: No slices found for Overall

In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=12),
      eval_args=tfx.proto.EvalArgs(num_steps=24))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## Run the pipeline

TFX supports multiple orchestrators to run pipelines.
In this tutorial we will use `LocalDagRunner` which is included in the TFX
Python package and runs pipelines on local environment.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

INFO:absl:Generating ephemeral wheel package for '/home/stud/s/ss490/share/bachelor_2022/artifact/tfrs_ranking_trainer.py' (including modules: ['tfrs_ranking_trainer']).
INFO:absl:User module package has hash fingerprint version bc2b229701436196f12efdbb4e880e3fa05103599208d98a05325f064775f21c.
INFO:absl:Executing: ['/opt/miniconda3/envs/rs_pipe/bin/python', '/tmp/tmpqfw9hi9g/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmphkfjt3st', '--dist-dir', '/tmp/tmpk5t2cvbx']
INFO:absl:Successfully built user code wheel distribution at 'pipeline/pipelines/TFRS-ranking/_wheels/tfx_user_code_Trainer-0.0+bc2b229701436196f12efdbb4e880e3fa05103599208d98a05325f064775f21c-py3-none-any.whl'; target user module is 'tfrs_ranking_trainer'.
INFO:absl:Full user module path is 'tfrs_ranking_trainer@pipeline/pipelines/TFRS-ranking/_wheels/tfx_user_code_Trainer-0.0+bc2b229701436196f12efdbb4e880e3fa05103599208d98a05325f064775f21c-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_spe

INFO:absl:Processing input csv data data/TFRS-ranking/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 3 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipeline/pipelines/TFRS-ranking/CsvExampleGen/examples/3"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:1979205,xor_checksum:1654179951,sum_checksum:1654179951"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "TFRS-ranking:2022-06-02T16:33:22.536612:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.4"
  }
}
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
 

Epoch 1/3
12/12 [==============================] - 3s 222ms/step - root_mean_squared_error: 2.0895 - loss: 4.1251 - regularization_loss: 0.0000e+00 - total_loss: 4.1251 - val_root_mean_squared_error: 1.1220 - val_loss: 1.2633 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.2633
Epoch 2/3
12/12 [==============================] - 3s 246ms/step - root_mean_squared_error: 1.1153 - loss: 1.2432 - regularization_loss: 0.0000e+00 - total_loss: 1.2432 - val_root_mean_squared_error: 1.1144 - val_loss: 1.2520 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.2520
Epoch 3/3
12/12 [==============================] - 2s 196ms/step - root_mean_squared_error: 1.1129 - loss: 1.2367 - regularization_loss: 0.0000e+00 - total_loss: 1.2367 - val_root_mean_squared_error: 1.1020 - val_loss: 1.1970 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.1970


INFO:tensorflow:Assets written to: pipeline/pipelines/TFRS-ranking/Trainer/model/4/Format-Serving/assets


INFO:tensorflow:Assets written to: pipeline/pipelines/TFRS-ranking/Trainer/model/4/Format-Serving/assets
INFO:absl:Training complete. Model written to pipeline/pipelines/TFRS-ranking/Trainer/model/4/Format-Serving. ModelRun written to pipeline/pipelines/TFRS-ranking/Trainer/model_run/4
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 4 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "pipeline/pipelines/TFRS-ranking/Trainer/model_run/4"
custom_properties {
  key: "name"
  value {
    string_value: "TFRS-ranking:2022-06-02T16:33:22.536612:Trainer:model_run:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.4"
  }
}
, artifact_type: name: "ModelRun"
)], 'model': [Artifact(artifact: uri: "pipeline/pipelines/TFRS-ranking/Trainer/model/4"
custom_properties {
  key: "name"
  value {
    string_value: "TFRS-ranking:2022-06-02T16:3

You should see "INFO:absl:Component Pusher is finished." at the end of the
logs if the pipeline finished successfully. Because `Pusher` component is the
last component of the pipeline.

The pusher component pushes the trained model to the `SERVING_MODEL_DIR` which
is the `serving_model/TFRS-ranking` directory if you did not change the
variables in the previous steps. You can see the result from the file browser
in the left-side panel in Colab, or using the following command:

In [ ]:
# List files in created model directory.
!ls -R {SERVING_MODEL_DIR}

pipeline/serving_model/TFRS-ranking:
1654180441

pipeline/serving_model/TFRS-ranking/1654180441:
assets	keras_metadata.pb  saved_model.pb  variables

pipeline/serving_model/TFRS-ranking/1654180441/assets:

pipeline/serving_model/TFRS-ranking/1654180441/variables:
variables.data-00000-of-00001  variables.index


Now we can test the ranking model by computing predictions for a user and a movie:

In [ ]:
import glob
# Load the latest model for testing
loaded = tf.saved_model.load(max(glob.glob(os.path.join(SERVING_MODEL_DIR, '*/')), key=os.path.getmtime))
print(loaded({'userId': [[42]], 'movieId': [[15]]}).numpy())

[[[3.5331185]]]


This concludes the TensorFlow Recommenders + TFX tutorial.